# Homework

In this homework, we'll deploy the ride duration model in batch mode. Like in homework 1, we'll use the Yellow Taxi Trip Records dataset. 

## Q1. Notebook

We'll start with the same notebook we ended up with in homework 1.
We cleaned it a little bit and kept only the scoring part. You can find the initial notebook [here](homework/starter.ipynb).

Run this notebook for the March 2023 data.

In [1]:
import pickle
import pandas as pd

with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')

    return df

df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)


What's the standard deviation of the predicted duration for this dataset?

In [3]:
float(round(y_pred.std(), 3))

6.247

## Q2. Preparing the output

Like in the course videos, we want to prepare the dataframe with the output. 

First, let's create an artificial `ride_id` column:

```python
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
```

Next, write the ride id and the predictions to a dataframe with results. 

Save it as parquet:

```python
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)
```

In [17]:
df_result = pd.DataFrame()
df_result['ride_id'] = f'{2023:04d}/{3:02d}_' + df.index.astype('str')
df_result['prediction'] = y_pred

output_file = 'output.parquet'
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)


What's the size of the output file?

__Note:__ Make sure you use the snippet above for saving the file. It should contain only these two columns. For this question, don't change the
dtypes of the columns and use `pyarrow`, not `fastparquet`. 

In [18]:
!du -h output.parquet

66M	output.parquet



## Q3. Creating the scoring script

Now let's turn the notebook into a script. 

Which command you need to execute for that?

In [20]:
!jupyter nbconvert --to script week4.ipynb

[NbConvertApp] Converting notebook week4.ipynb to script
[NbConvertApp] Writing 5624 bytes to week4.py


## Q4. Virtual environment

Now let's put everything into a virtual environment. We'll use pipenv for that.

Install all the required libraries. Pay attention to the Scikit-Learn version: it should be the same as in the starter
notebook.

After installing the libraries, pipenv creates two files: `Pipfile`
and `Pipfile.lock`. The `Pipfile.lock` file keeps the hashes of the
dependencies we use for the virtual env.

What's the first hash for the Scikit-Learn dependency?

In [21]:
import json

with open('Pipfile.lock', 'r') as file:
    data = json.load(file)

scikit_learn_data = data['default']['scikit-learn']['hashes'][0]
print(f"First hash for scikit-learn: {scikit_learn_data}")

First hash for scikit-learn: sha256:057b991ac64b3e75c9c04b5f9395eaf19a6179244c089afdebaad98264bff37c


## Q5. Parametrize the script

Let's now make the script configurable via CLI. We'll create two 
parameters: year and month.

Run the script for April 2023. 

What's the mean predicted duration?

Hint: just add a print statement to your script.

In [23]:
!python week4.py 2023 4

The mean predicted duration: 14.292


## Q6. Docker container 

Finally, we'll package the script in the docker container. 
For that, you'll need to use a base image that we prepared. 

This is what the content of this image is:

```dockerfile
FROM python:3.10.13-slim

WORKDIR /app
COPY [ "model2.bin", "model.bin" ]
```

Note: you don't need to run it. We have already done it.

It is pushed to [`agrigorev/zoomcamp-model:mlops-2024-3.10.13-slim`](https://hub.docker.com/layers/agrigorev/zoomcamp-model/mlops-2024-3.10.13-slim/images/sha256-f54535b73a8c3ef91967d5588de57d4e251b22addcbbfb6e71304a91c1c7027f?context=repo),
which you need to use as your base image.

That is, your Dockerfile should start with:

```dockerfile
FROM agrigorev/zoomcamp-model:mlops-2024-3.10.13-slim

# do stuff here
```

This image already has a pickle file with a dictionary vectorizer
and a model. You will need to use them.

Important: don't copy the model to the docker image. You will need
to use the pickle file already in the image. 

In [26]:
!docker build -t mlops-course-week4 .

[+] Building 0.0s (0/0)  docker:default
[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.2s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 276B                                       0.0s
 => [internal] load metadata for docker.io/agrigorev/zoomcamp-model:mlops  0.2s
[+] Building 0.3s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 276B                                       0.0s
 => [internal] load metadata for docker.io/agrigorev/zoomcamp-model:mlops  0.3s
[+] Building 0.5s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 276B                                       0.0s



Now run the script with docker. 

What's the mean predicted duration for May 2023?

In [27]:
!docker run -it --rm docker.io/library/mlops-course-week4 2023 5

The mean predicted duration: 0.192


## Summary

### Q1. Notebook

What's the standard deviation of the predicted duration for this dataset?

`6.24`

### Q2. Preparing the output

What's the size of the output file?

`66M`

### Q3. Creating the scoring script

Which command you need to execute for that?

`jupyter nbconvert --to script week4.ipynb`

### Q4. Virtual environment

What's the first hash for the Scikit-Learn dependency?

`sha256:057b991ac64b3e75c9c04b5f9395eaf19a6179244c089afdebaad98264bff37c`

### Q5. Parametrize the script

What's the mean predicted duration? 

`14.29`

### Q6. Docker container

What's the mean predicted duration for May 2023? 

`0.19`